In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from modules import *

In [ ]:
data = pd.read_csv("막걸리.csv", encoding='utf-8-sig')
data.drop_duplicates(['text'],ignore_index=True, inplace=True) # 중복 제거
data.columns = ['date', 'star','doc']

In [ ]:
data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
data.doc = data.doc.str.replace("\s+", " ") # white space 삭제

data.doc

0                                     원래도 조아래요 배송오니 너무 좋아요
1        자주 주문해서 먹고 있습니다믿고먹는 느린마을내일을 위해 조금 남겨뒀는데남은 것도 잘...
2                  밀키스 마시는거 같아요 목넘김이 좋아서 그냥 들어가네요 금방 취합니다 
3                            우유같은부드러움에 맛도좋고가격도좋고요즘막걸리즐겨마셔요
4              지인분 선물로 보내드렸는데 첨 마셔본건데 너무 맛있다고 만족하셨어요 감사합니다
                               ...                        
18919                                           아버지가 좋아하세요
18920                                           병짜리 생겨 좋아요
18921                                         역시 느린민을아주좋인요
18922                                                     
18923    배송중 찌그러져 조금 아쉬웠지만 첨가물이 없는 막걸리라 숙취가 없어 좋아요냉장고에 ...
Name: doc, Length: 18924, dtype: object

In [ ]:
# 검토 후 추가 삭제 문구 적용
data.dropna(inplace=True)

In [ ]:
data[data.doc.isnull()]

,date,star,doc


In [ ]:
# data = data[data.doc.str.contains('맛|향|달|넘김|부드|단|도수|냄세')]
# data.reset_index(drop=True, inplace=True)

In [ ]:
len(data)

18923

In [ ]:
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize
from konlpy.tag import Okt

In [ ]:
#원하는 태그 추출 & 불용어 제거
okt = Okt()

#불용어
stopwords = pd.read_csv('ko-stopwords.csv') #한국어불용어사전 불러오기
stopwords=list(stopwords['stopwords']) 
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다",
                 '에서','고','이다','쓰리다 도어즈','일요일','휴무','수표','층','틀다','차차','드므','맛집','술집','길',
                 '층','개다 성','사진','파다','동영상','아스 론','자다','는','희다 스토리','한','씨', "것","거","게","데",
                 "이다","건","고","되다","되어다","걸","기", "시","네","듯","랍니","중이","얘","스","도도", "나","수","개",
                 "내","기","제","저","인","있다","이렇다", "그렇다","번","위","팅","분","인","링","란","포","두", "진짜",
                 "하다" ,"이다" ,"가다", "이제" ,"들다", "먹다", "보다"]) #불용어 추가

stopwords=set(stopwords) # 중복제거    

# 형태소 분석 함수 만들기
def okt_pos_tagging2(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]
    word2=[]
    word2.append(words)
    bigr = bigram_(word2)
    asd = []
    
    for i in range(len(bigr[2])):
        asd.append(bigr[2][i])

    return asd


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(tokenizer = lambda x: okt_pos_tagging(x))
bow_vect = vect.fit_transform(data['doc'].tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)

In [ ]:
bow_vect.shape

(18923, 6137)

In [ ]:
word_count_dict = dict(zip(word_list, count_list))
word_count_dict


{'가가': 5,
 '가게': 15,
 '가격': 279,
 '가공': 1,
 '가구': 2,
 '가기': 15,
 '가까이': 2,
 '가깝다': 8,
 '가끔': 99,
 '가나': 4,
 '가나다': 1,
 '가누다': 1,
 '가능': 4,
 '가능하다': 24,
 '가다리': 1,
 '가대': 1,
 '가도': 4,
 '가득': 3,
 '가득하다': 9,
 '가뜩이나': 1,
 '가라앉다': 11,
 '가라앉히다': 2,
 '가량': 1,
 '가로': 1,
 '가루': 14,
 '가르다': 3,
 '가르침': 1,
 '가리다': 2,
 '가리비': 1,
 '가마': 1,
 '가맹': 1,
 '가면': 18,
 '가무': 2,
 '가물': 20,
 '가미': 6,
 '가방': 5,
 '가볍다': 74,
 '가보': 1,
 '가보다': 19,
 '가성': 70,
 '가세': 1,
 '가수': 1,
 '가쉽': 1,
 '가스': 9,
 '가슴': 2,
 '가시다': 2,
 '가안': 4,
 '가야': 4,
 '가오': 1,
 '가요': 19,
 '가운': 1,
 '가원': 1,
 '가을': 102,
 '가을로': 4,
 '가이거': 1,
 '가이드': 1,
 '가입': 1,
 '가장': 115,
 '가정': 1,
 '가제': 2,
 '가져가다': 16,
 '가져오다': 4,
 '가조': 1,
 '가족': 162,
 '가즈': 1,
 '가지': 18,
 '가지다': 5,
 '가지런하다': 1,
 '가첨': 1,
 '가치': 2,
 '가터': 1,
 '가하다': 3,
 '가형': 1,
 '가흥': 1,
 '각기': 1,
 '각오': 1,
 '간': 5,
 '간격': 11,
 '간다': 4,
 '간단': 2,
 '간단하다': 12,
 '간만': 13,
 '간문제': 1,
 '간지': 1,
 '간지나다': 1,
 '간편하다': 8,
 '간혹': 8,
 '갇': 1,
 '갇습니': 1,
 '갈다': 48,
 '갈리': 1,
 '갈리다': 10,
 '갈릴': 4,
 

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

In [ ]:
print(tf_idf_vect.shape)


(18923, 6137)


In [ ]:
print(tf_idf_vect[0])

  (0, 4829)	0.16131275387591024
  (0, 4093)	0.42888687617009025
  (0, 3857)	0.5799591759961854
  (0, 3429)	0.640669346240603
  (0, 2318)	0.2079047518732931


In [ ]:
invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
print(str(invert_index_vectorizer)[:100]+'...')

{4093: '원래', 3429: '아래', 2318: '배송', 3857: '오니', 4829: '좋다', 4475: '자주', 4867: '주문', 2185: '믿다', 102...


In [ ]:
def rating_to_label(rating):
    if rating > 3:
        return 1
    else:
        return 0

data['y'] = data['star'].apply(lambda x : rating_to_label(x))
len(data[data['y']==0])

630

In [ ]:
from sklearn.model_selection import train_test_split

x = tf_idf_vect
y = data['y']

# positive_random_idx = data[data['y']==1].sample(48, random_state=12).index.tolist()
# negative_random_idx = data[data['y']==0].sample(48, random_state=12).index.tolist()

# random_idx = positive_random_idx + negative_random_idx
# x = tf_idf_vect[random_idx]
# y = data['y'][random_idx]


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=1)


In [ ]:
x.shape,y.shape

((18923, 6137), (18923,))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# fit in training set
lr = LogisticRegression()
lr.fit(x_train, y_train)

# predict in test set
y_pred = lr.predict(x_test)

In [ ]:
print('accuracy: %.4f' % accuracy_score(y_test, y_pred))
print('precision: %.4f' % precision_score(y_test, y_pred))
print('recall: %.4f' % recall_score(y_test, y_pred))
print('F1: %.4f' % f1_score(y_test, y_pred))


accuracy: 0.9658
precision: 0.9660
recall: 0.9998
F1: 0.9826


In [ ]:
coef_pos_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = True)
coef_neg_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = False)
coef_pos_index



[(2.413687611520334, 1842),
 (2.0623190558196915, 5881),
 (2.0132920958277953, 4867),
 (1.8510049985685355, 4839),
 (1.8275451017044986, 5276),
 (1.5351564074505293, 1733),
 (1.4997804679674978, 1046),
 (1.442760833121237, 4829),
 (1.4190363606156173, 3785),
 (1.409688769837935, 4563),
 (1.2465753986756738, 673),
 (1.2417777194331334, 2519),
 (1.2049873456723537, 1023),
 (1.0923052892971754, 1780),
 (1.076616089396499, 4742),
 (1.0754288480909076, 1716),
 (1.0701796204092484, 4330),
 (1.0593908994068044, 4433),
 (1.0416479675752315, 497),
 (1.0261800365085416, 5289),
 (0.9883480933999517, 5008),
 (0.9661600509484679, 5835),
 (0.9375614901139492, 909),
 (0.934240175082637, 1344),
 (0.9214116934569295, 1705),
 (0.9114600193710903, 1741),
 (0.8767096644960461, 717),
 (0.8681353538416307, 3468),
 (0.8647844813763873, 3934),
 (0.8587722580230053, 3249),
 (0.8528211083720106, 305),
 (0.8436237136840741, 4694),
 (0.822784760115252, 3475),
 (0.7748855467797889, 4475),
 (0.7739255174623636, 191

In [ ]:
for coef in coef_pos_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])


맛있다 2.413687611520334
항상 2.0623190558196915
주문 2.0132920958277953
좋아하다 1.8510049985685355
최고 1.8275451017044986
마트 1.5351564074505293
늘 1.4997804679674978
좋다 1.442760833121237
역시 1.4190363606156173
재다 1.409688769837935
깔끔하다 1.2465753986756738
부드럽다 1.2417777194331334
느리다 1.2049873456723537
말다 1.0923052892971754
제일 1.076616089396499
마을 1.0754288480909076
인생 1.0701796204092484
잇다 1.0593908994068044
굿 1.0416479675752315
최애 1.0261800365085416


In [ ]:
for coef in coef_neg_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])


그냥 -2.6715852831539193
달다 -2.5874644712114216
느끼하다 -2.546724364164931
싱겁다 -2.0228137033839655
세다 -1.8825384055827323
새다 -1.768385060941123
제입 -1.6611483213678557
전혀 -1.6200602195331677
모르다 -1.6163483316043488
별로 -1.6045147106870419
맞다 -1.5924252403011743
입맛 -1.5862637029331748
보통 -1.5583587293523404
밍밍 -1.488515114321043
아쉽다 -1.4675393410535404
아니다 -1.4513067378796254
강하다 -1.4094521609851764
취향 -1.352424623279607
버리다 -1.3433133257412175
기대하다 -1.3302069426276049


In [ ]:
for coef in coef_neg_index[:]:
    print(invert_index_vectorizer[coef[1]], coef[0])

그냥 -2.6715852831539193
달다 -2.5874644712114216
느끼하다 -2.546724364164931
싱겁다 -2.0228137033839655
세다 -1.8825384055827323
새다 -1.768385060941123
제입 -1.6611483213678557
전혀 -1.6200602195331677
모르다 -1.6163483316043488
별로 -1.6045147106870419
맞다 -1.5924252403011743
입맛 -1.5862637029331748
보통 -1.5583587293523404
밍밍 -1.488515114321043
아쉽다 -1.4675393410535404
아니다 -1.4513067378796254
강하다 -1.4094521609851764
취향 -1.352424623279607
버리다 -1.3433133257412175
기대하다 -1.3302069426276049
줄줄 -1.286098511636306
터지다 -1.265332429796333
부족하다 -1.2457047184123333
생각 -1.2424466141359898
심심하다 -1.2255242593026985
늘다 -1.1851935474245372
안 -1.1349183264286768
냄새 -1.1115098292715841
밋밋하다 -1.0732851724798922
넣다 -1.0339352405492248
리뷰 -1.0327728147597617
굿굿굿 -1.0294966571957949
병 -1.007222803272269
사이트 -1.0041723841741532
프라스 -0.9881323494248269
신맛 -0.9821084354421602
별 -0.9631158386290063
막다 -0.9510649034397745
허다 -0.9502834414581471
문제 -0.9489447096204218
식품 -0.9425287602148644
음료 -0.9387137212781954
꿋꿌 -0.9010205216386169
실